# Capstone Project

###### This set of code will help you figure out whether which place is crowded or empty and most ideal to start a bussiness

Importing libraries

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
                                                            
!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.



WARNING conda.base.context:use_only_tar_bz2(632): Conda is constrained to only using the old .tar.bz2 file format because you have conda-build installed, and it is <3.18.3.  Update or remove conda-build to get smaller downloads and faster extractions.


Solving environment: ...working... done

# All requested packages already installed.



WARNING conda.base.context:use_only_tar_bz2(632): Conda is constrained to only using the old .tar.bz2 file format because you have conda-build installed, and it is <3.18.3.  Update or remove conda-build to get smaller downloads and faster extractions.


Folium installed
Libraries imported.


__________________
###### Initializing variables to be passed to the Foursquare API 

In [2]:
CLIENT_ID = 'WZD0TXK450FDKF3OPPP4NJS5AZHYQNERPWK11Q3C311PGG3F' # your Foursquare ID
CLIENT_SECRET = 'UXWCVABM3AMICU0W4IPGTIUBQIPC2KX5WIDUZFCUWBGGURD5' # your Foursquare Secret
VERSION = '20180604'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WZD0TXK450FDKF3OPPP4NJS5AZHYQNERPWK11Q3C311PGG3F
CLIENT_SECRET:UXWCVABM3AMICU0W4IPGTIUBQIPC2KX5WIDUZFCUWBGGURD5


___________________________________
###### Get the Latitudes and Longitudes of the Area

In [121]:
address = 'Mumbai, IN'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

*****************
###### Making up the URL and setting up the Dataframe

In [141]:
search_query = 'food'
radius = 50000
LIMIT = 5000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=WZD0TXK450FDKF3OPPP4NJS5AZHYQNERPWK11Q3C311PGG3F&client_secret=UXWCVABM3AMICU0W4IPGTIUBQIPC2KX5WIDUZFCUWBGGURD5&ll=18.9387711,72.8353355&v=20180604&query=food&radius=50000&limit=5000'

In [ ]:
results = requests.get(url).json()
results

In [143]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1c7941735', 'name': 'S...",False,5097a524498ecb7e5608a872,NaN,IN,NaN,India,NaN,217,[India],"[{'label': 'display', 'lat': 18.94072314746390...",18.940723,72.835401,NaN,NaN,NaN,"Refresh Food Plaza, CST",v-1562101885,NaN
1,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",False,4cfe22077f2db1f745873bd4,"@Kitabkhana, Somaiya Bhavan",IN,Mumbai,India,"45/47 Mahatma Gandhi Road, Fort",843,"[@Kitabkhana, Somaiya Bhavan (45/47 Mahatma G...","[{'label': 'display', 'lat': 18.93203054810475...",18.932031,72.831667,NaN,400001,Mahārāshtra,Food for Thought,v-1562101885,33013527
2,"[{'id': '50aa9e744b90af0d42d5de0e', 'name': 'H...",False,5a06b0a77269fe0a9a40b515,Mumbai Maharashtra 400078 India,IN,Mumbai,India,NaN,629,"[Mumbai Maharashtra 400078 India, Mumbai 40007...","[{'label': 'display', 'lat': 18.93316157126705...",18.933162,72.834570,NaN,400078,Mahārāshtra,Food supplements online,v-1562101885,NaN
3,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",False,4e6a0ba5483bf2d9e609c9ac,NaN,IN,Mumbai,India,NaN,364,"[Mumbai, Mahārāshtra, India]","[{'label': 'display', 'lat': 18.94197843351325...",18.941978,72.834649,NaN,NaN,Mahārāshtra,Times Food Court,v-1562101885,NaN
4,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",False,53803a2d498e5da6507c877d,NaN,IN,NaN,India,NaN,366,[India],"[{'label': 'display', 'lat': 18.94195069611220...",18.941951,72.834451,NaN,NaN,NaN,BBC Good Food Magazine,v-1562101885,NaN


********************
###### Cleaning the DataFrame further

In [144]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.shape

(50, 16)

In [145]:
dataframe_filtered = dataframe_filtered[['name', 'lat', 'lng', 'id']]

In [146]:
#dataframe_filtered = dataframe_filtered.dropna()
data = dataframe_filtered

*************
###### Creating the final MAP which will be used for Figuring out our goal

In [147]:
# create map of New York using latitude and longitude values
map_mumbai = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label_ in zip(data['lat'], data['lng'], data['name']):
    label_ = folium.Popup(label_, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label_,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_mumbai)  
    
map_mumbai